In [4]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import json
from dateutil import parser
import datetime as dt
import numpy as np
import pprint
import matplotlib.pyplot as plt

In [5]:
df = pd.DataFrame
analyzer = SentimentIntensityAnalyzer()
d_start = dt.date(2017,12,13)
d_end = dt.date(2018,3,19)

In [6]:
tweets = pd.read_json(open('twitter archive'))
tweets = tweets[tweets.created_at >= dt.date(2017,12,13)]

In [7]:
tweets.head()

,created_at,favorite_count,id_str,is_retweet,retweet_count,source,text
0,2018-03-20 00:28:45,21050,975891861863829504,False,5457,Twitter for iPhone,The Democrats do not want to help DACA. Would ...
1,2018-03-19 22:38:15,0,975864052017659904,True,4537,Twitter for iPhone,RT @WhiteHouse: The opioid crisis has created ...
2,2018-03-19 13:07:50,83709,975720503997620224,False,19726,Twitter for iPhone,A total WITCH HUNT with massive conflicts of i...
3,2018-03-19 12:16:40,37803,975707627182219264,False,6619,Twitter for iPhone,.@seanhannity on @foxandfriends now! Great! 8:...
4,2018-03-18 12:35:42,112576,975350027169206272,False,30561,Twitter for iPhone,Why does the Mueller team have 13 hardened Dem...


In [8]:
t_sent = list()
for tweet in tweets['text']:
    t_sent.append(analyzer.polarity_scores(tweet)['compound'])
tweets['sentiment'] = t_sent

In [9]:
tweets.head()

,created_at,favorite_count,id_str,is_retweet,retweet_count,source,text,sentiment
0,2018-03-20 00:28:45,21050,975891861863829504,False,5457,Twitter for iPhone,The Democrats do not want to help DACA. Would ...,0.3722
1,2018-03-19 22:38:15,0,975864052017659904,True,4537,Twitter for iPhone,RT @WhiteHouse: The opioid crisis has created ...,0.0900
2,2018-03-19 13:07:50,83709,975720503997620224,False,19726,Twitter for iPhone,A total WITCH HUNT with massive conflicts of i...,-0.4810
3,2018-03-19 12:16:40,37803,975707627182219264,False,6619,Twitter for iPhone,.@seanhannity on @foxandfriends now! Great! 8:...,0.6892
4,2018-03-18 12:35:42,112576,975350027169206272,False,30561,Twitter for iPhone,Why does the Mueller team have 13 hardened Dem...,0.4440


In [10]:
n_date = list()
n_vol = list()
a_sent = list()
a_source = list()
newsdf = df()

In [11]:
for ds in range(d_start.toordinal(), d_end.toordinal()) :
    ds = dt.date.fromordinal(ds)
    n_date.append(ds)
    news = json.load(open('download/trump_'+ds.isoformat()+'.json'))
    n_vol.append(news['totalResults'])
    newsdf = pd.concat([newsdf,df(news['articles'])])
    for article in news['articles'] :
        header = article['title']
        desc = article['description']
        a_source.append(article['source']['name'])
        a_sent.append(analyzer.polarity_scores(header+desc)['compound'])

In [12]:
newsdf['sentiment'] = a_sent
newsdf['source'] = a_source

In [ ]:
tweets.to_csv('tweets.csv', encoding='utf-8')
newsdf.to_csv('news.csv', encoding='utf-8')
news_agg = df({'date' : n_date, 'volume' : n_vol})
news_agg.to_csv('news_agg.csv', encoding='utf-8')

In [ ]:
tweets.to_csv('tweets.csv', sep=',',encoding='utf-8')

In [13]:
tweets['type'] = 'tweet'

In [14]:
newsdf['type'] = 'news'
newsdf.head()

,author,description,publishedAt,source,title,url,urlToImage,sentiment,type
0,Jon Fingas,The US' brief period of registration-free dron...,2017-12-13T00:44:00Z,Engadget,US once again requires that you register your ...,https://www.engadget.com/2017/12/12/trump-sign...,https://o.aolcdn.com/images/dims?thumbnail=120...,0.4767,news
1,Sandra Upson,"Artificial intelligence-generated video, photo...",2017-12-13T12:00:00Z,Wired,Artificial Intelligence Is Killing the Uncanny...,https://www.wired.com/story/future-of-artifici...,https://media.wired.com/photos/5a2eb8b77ddf3c6...,-0.3182,news
2,Tom Warren,pew pew,2017-12-13T12:31:17Z,The Verge,Apple invests $390 million in US-made iPhone l...,https://www.theverge.com/2017/12/13/16771246/a...,https://cdn.vox-cdn.com/thumbor/rI0jDb-MMcxMBl...,0.0000,news
3,Dell Cameron,Plagued by infighting and the constant threat ...,2017-12-13T19:35:00Z,Gizmodo.com,"After Alabama Defeat, Republicans Should Think...",https://gizmodo.com/after-alabama-defeat-repub...,https://i.kinja-img.com/gawker-media/image/upl...,-0.7783,news
4,RICHARD FAUSSET,"Turned off by Roy S. Moore, Stephen K. Bannon ...",2017-12-13T14:04:44Z,The New York Times,Why Black Voters Backed Doug Jones: ‘It’s a Ma...,https://www.nytimes.com/2017/12/13/us/doug-jon...,https://static01.nyt.com/images/2017/12/13/us/...,0.0516,news


In [17]:
tweets['publishedAt'] = tweets['created_at']
combined = pd.concat([newsdf[['publishedAt','sentiment','type']], tweets[['publishedAt','sentiment','type']]])

In [18]:
combined.head()

,publishedAt,sentiment,type
0,2017-12-13T00:44:00Z,0.4767,news
1,2017-12-13T12:00:00Z,-0.3182,news
2,2017-12-13T12:31:17Z,0.0000,news
3,2017-12-13T19:35:00Z,-0.7783,news
4,2017-12-13T14:04:44Z,0.0516,news


In [19]:
combined.to_csv('combined.csv', sep=',',encoding='utf-8')